In [19]:
# Load CIFAR-10 dataset
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [20]:
# Assume class 0 is the minority class
import numpy as np
minority_class_label = 0
minority_indices_train = np.where(y_train == minority_class_label)[0]
num_indices_to_keep=len(minority_indices_train) // 2
indices_to_keep_train = minority_indices_train[:num_indices_to_keep]
# Remove indices corresponding to minority class label from y_train
y_train_filtered = np.delete(y_train, indices_to_keep_train, axis=0)
x_train_filtered = np.delete(x_train, indices_to_keep_train, axis=0)

In [21]:
minority_indices_train = np.where(y_test == minority_class_label)[0]
num_indices_to_keep=len(minority_indices_train) // 2
indices_to_keep_train = minority_indices_train[:num_indices_to_keep]
# Remove indices corresponding to minority class label from y_train
y_train_filtered = np.delete(y_test, indices_to_keep_train, axis=0)
x_train_filtered = np.delete(x_test, indices_to_keep_train, axis=0)

array([   29,    30,    35, ..., 25070, 25082, 25083], dtype=int64)

In [13]:
len(x_train_filtered)

47500